# Project 2 Write Up

  Image Segmentation  
  CS601R Spring Semester  
  Written by Clayton Ramstedt  

## 1.1 Model

Throughout the project I used resnet50 as a base with five deconvolution layers replacing the final resnet layer. The configuration of these layers can be seen below:
```python
self.deconv1 = nn.ConvTranspose2d(2048, 512, 3, stride=2, padding=1, output_padding=1)
self.deconv2 = nn.ConvTranspose2d(512, 128, 4, stride=2, padding=1, output_padding=1)
self.deconv3 = nn.ConvTranspose2d(128, 64, 5, stride=2, padding=2, output_padding=1)
self.deconv4 = nn.ConvTranspose2d(64, 32, 4, stride=2, padding=3, output_padding=1)
self.deconv5 = nn.ConvTranspose2d(32, 7, 4, stride=2, padding=2, output_padding=1)
nn.init.xavier_uniform_(self.deconv1.weight)
nn.init.xavier_uniform_(self.deconv2.weight)
nn.init.xavier_uniform_(self.deconv3.weight)
nn.init.xavier_uniform_(self.deconv4.weight)
nn.init.xavier_uniform_(self.deconv5.weight)
self.deconvRelu1 = nn.LeakyReLU()
self.deconvRelu2 = nn.LeakyReLU()
self.deconvRelu3 = nn.LeakyReLU()
self.deconvRelu4 = nn.LeakyReLU()
self.deconvRelu5 = nn.LeakyReLU()
```
The purpose of this model was to be able to take in a 224x224 image and output a segmentation map for each pixel. Because I had no prior experience with convolutional networks, getting the dimensions right on the deconvolution layers was a trial in and of itself. And so once I got a working model, I stopped playing with. However, if I were to revisit this, I would like to experiment with the kernel sizes.

The only change I made to the model after version one, was to change the `ReLU()` to the `LeakyReLU()` based on a suggestion given in the class Slack. Unfortunately when I trained the new model, I had also made changes to how the dataset was being generated, so I am unsure what impact, if any, this change had.

*For more information on the complete model, see `SegNet.py`*

## 1.2 Dataset

Over the course of the project, the way that the dataset was interpreted evolved the most, and is primarily responsible for any performance gains I had. Because I had no idea what I was doing, the original implementation used a naive rescaling of all images to 224x224. I trained a single model only on this type of data and obtained an average IOU score across all the classes of `0.55`. It wasn't great, but it was a good starting point!

Version two first checked to see if 

In [1]:
%matplotlib inline

TRAIN_DATA = "../mscoco_subset_cs601r/train/"
TRAIN_DATA_LABEL = "../mscoco_subset_cs601r/train_masks/"
TEST_DATA = "../mscoco_subset_cs601r/test/"
TEST_DATA_LABEL = "../mscoco_subset_cs601r/test_masks/"

In [2]:
import torch.nn as nn
import torchvision.models as models
from Data import Data
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import pickle
import numpy as np
from SegNet import SegNet
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import cv2